In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import

In [105]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from bs4 import BeautifulSoup
import nltk
import unidecode
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import emoji
import re

In [106]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/it_core_news_sm-3.2.0/it_core_news_sm-3.2.0.tar.gz
!pip install unidecode
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.4 MB 2.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
#Downloads for stopwords and punctuation
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('italian')
nltk.download('wordnet')
string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [108]:
# likes_path = "easytour-likes-2022-03MAR-21.json"
# schede_path = "easytour-schede-2022-03MAR-21.json"
#likes_path = "kuriu-likes-apr.json"
#schede_path = "kuriu-schede-apr.json"
likes_path = "/content/drive/MyDrive/Tesi/kuriu-likes-2022-06JUN-18.json"
schede_path = "/content/drive/MyDrive/Tesi/kuriu-schede-2022-06JUN-18.json"
data_path = "/content/drive/MyDrive/Tesi/data-JUN.csv"
data_analyzed_path = "/content/drive/MyDrive/Tesi/data_analyzed-JUN.csv"

In [109]:
f_schede = open(schede_path)
data_schede = json.load(f_schede)
print("Fields: {}".format(data_schede[0].keys())) 
print("Number of samples: {}".format(len(data_schede)))

Fields: dict_keys(['id', 'userId', 'userName', 'documentType', 'title', 'description', 'locations', 'duration', 'imageId', 'previewId', 'distance', 'datePublishing', 'widgets', 'counters', 'features', 'tags', 'numberOfPositions', 'categories', 'videoId', 'creationDate', 'version', 'media', 'shareType', 'status', 'lastUpdate', 'versionHistory', 'viewCounter', 'device', 'crowdPosting', 'activeCrowdPosting'])
Number of samples: 5724


In [110]:
print("First sample: {}".format(data_schede[8]))

First sample: {'id': '621128704f9d177c90f36d8a', 'userId': 579, 'userName': 'laviadellescimmie', 'documentType': 'experience', 'title': 'The Lion Rock, l’antica città di Sigiriya', 'description': 'Una delle attrazioni più suggestive di tutto lo Sri Lanka è la Lion Rock. Questa si trova all’interno di un parco chiamato Sigiriya. Si tratta di un’enorme roccia che svetta sulla vallata, immersa nella vegetazione. Si parla anche dell’antica città di Sigiriya perché sulla sua sommità si possono ancora ammirare le rovine di un antico palazzo (della fine del V secolo) e le zone coltivate. Sigiriya rientra nei patrimoni dell’umanità dello Sri Lanka. \n\nPer evitare di imbattersi in un gran numero di turisti, è meglio visitare il sito archeologico alla mattina, all’apertura. La luce è migliore, il clima è clemente e ci sono meno persone. La roccia deriva dall’erosione di un antico vulcano, ecco perché la sua forma è così particolare e unica in tutto il Paese. La roccia si trova su una collina al

In [111]:
f_likes = open(likes_path)
data_likes = json.load(f_likes)
print("Number of samples: {}".format(len(data_likes)))
print("Fields: {}".format(data_likes[0].keys()))
print("First sample: {}".format(data_likes[0]))

Number of samples: 2542
Fields: dict_keys(['documentId', 'lastUpdate', 'authorId', 'userName', 'type'])
First sample: {'documentId': '622c3b8ac992ca28f33c7f21', 'lastUpdate': '2022-03-19T07:56:20.868Z', 'authorId': 606, 'userName': 'DaleOnTravel', 'type': 'Useful'}


In [112]:
df_schede = pd.read_json(schede_path)
df_likes = pd.read_json(likes_path)

#Data Analysis

In [113]:
df_schede.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5724 entries, 0 to 5723
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5724 non-null   object 
 1   userId              5724 non-null   int64  
 2   userName            5724 non-null   object 
 3   documentType        5724 non-null   object 
 4   title               5724 non-null   object 
 5   description         5724 non-null   object 
 6   locations           5724 non-null   object 
 7   duration            5724 non-null   int64  
 8   imageId             5723 non-null   object 
 9   previewId           598 non-null    object 
 10  distance            5724 non-null   int64  
 11  datePublishing      5713 non-null   object 
 12  widgets             4788 non-null   object 
 13  counters            5724 non-null   object 
 14  features            5724 non-null   object 
 15  tags                5724 non-null   object 
 16  number

In [114]:
df_schede.describe()

,userId,duration,distance,numberOfPositions,version,viewCounter,crowdPosting
count,5724.000000,5.724000e+03,5724.0,5724.000000,5724.000000,5724.000000,0.0
mean,458.611286,1.848888e+07,0.0,1.300314,6.140985,41.306953,NaN
std,151.017776,6.209633e+07,0.0,1.000328,9.479419,42.832072,NaN
min,77.000000,0.000000e+00,0.0,1.000000,2.000000,1.000000,NaN
25%,314.000000,0.000000e+00,0.0,1.000000,3.000000,15.000000,NaN
50%,498.000000,3.600000e+06,0.0,1.000000,4.000000,45.000000,NaN
75%,572.500000,1.080000e+07,0.0,1.000000,7.000000,59.000000,NaN
max,708.000000,1.296000e+09,0.0,12.000000,414.000000,2134.000000,NaN


In [115]:
df_likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   documentId  2542 non-null   object
 1   lastUpdate  2542 non-null   object
 2   authorId    2542 non-null   int64 
 3   userName    2542 non-null   object
 4   type        2542 non-null   object
dtypes: int64(1), object(4)
memory usage: 99.4+ KB


In [116]:
#df_schede_fields = df_schede[["id","userId", "title", "description", "duration", "datePublishing", "creationDate", "counterUseful", "categories", "viewCounter"]]
df_schede_fields = df_schede[["id","userId", "title", "description", "duration", "datePublishing", "creationDate", "categories", "viewCounter"]]
#df_likes_ids = df_likes[["id", "userId"]]
df_likes_ids = df_likes[["documentId", "authorId"]]

In [117]:
df_schede_fields

,id,userId,title,description,duration,datePublishing,creationDate,categories,viewCounter
0,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,[Viaggi],18
1,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,[Viaggi],16
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,[Viaggi],426
3,624e5982ec9ff859ff529ce8,571,"Yangon, la capitale storica del Myanmar",Probabilmente Yangon sarà il punto d’inizio se...,0,2022-04-07T03:24:51.101Z,2022-04-07T03:24:50.000+00:00,[Viaggi],6
4,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,18000000,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,[Viaggi],424
...,...,...,...,...,...,...,...,...,...
5719,6213b3460f41f86438b07d9c,429,"Machu Picchu, meraviglia del mondo","È l’alba, ma la realtà è che in questo viaggio...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,[Viaggi],47
5720,61dd867f9c823e129b1f2cc2,429,I segreti di Moray,Sembra la prova schiacciante dell’esistenza de...,7200000,2022-01-11T13:30:45.997Z,2022-01-11T13:30:39.000+00:00,[Aria aperta],95
5721,61e0146111b3bf4d3185ad5d,427,"Machu Picchu, tra storia e meraviglia",Sono passati dieci (!) anni dalla mia esperien...,16200000,2022-01-13T12:04:35.591Z,2022-01-13T12:00:33.000+00:00,[Cultura],63
5722,6273a0a9dd67f6198e5ac192,606,Come arrivare a Machu Picchu: modalità e tempi,Sembrerà un argomento banale: vuoi che sia dif...,0,2022-05-05T10:02:17.841Z,2022-05-05T10:02:17.000+00:00,[Viaggi],7


#Preprocessing

##Null or empty values

In [118]:
#Check if there are null fields
df_schede_fields.isnull().sum()

id                 0
userId             0
title              0
description        0
duration           0
datePublishing    11
creationDate       0
categories         1
viewCounter        0
dtype: int64

In [119]:
def fill_null_fields(df):
    #empty description -> remove or use '' ?
    df["description"].fillna('', inplace=True)
    #if duration is null -> set to 0
    df["duration"].fillna(0, inplace=True)
    #if datePublishing is null -> set it equal to creationDate
    df["datePublishing"].fillna(df_schede_fields.creationDate, inplace=True)
    #if viewCounter is null -> set to 0
    df["viewCounter"].fillna(0, inplace=True)

In [120]:
fill_null_fields(df_schede_fields)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


##Categories into one hot encoding

In [121]:
def lower_list(l):
    if type(l)!=list:
        return []
    res = [x.lower() for x in l]
    return res

def encoding_categories(df):
  one_hot_enc_categories = df["categories"].apply(lambda x: lower_list(x))
  df["categories_encoding"] = one_hot_enc_categories
  mlb = MultiLabelBinarizer()

  schede_with_categories = df.join(
              pd.DataFrame(
                  mlb.fit_transform(df.pop('categories_encoding')),
                  index=df.index,
                  columns=mlb.classes_))
  return schede_with_categories

In [122]:
df_categories = encoding_categories(df_schede_fields)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


##Text fields

In [123]:
nlp = spacy.load("it_core_news_sm")

def keep_just_nouns(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_=="NOUN"]

def remove_verbs_adj(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_ not in ["VERB", "ADJ", "ADV"]]

In [124]:
def demojize(field):
    try:
        result = emoji.demojize(field)
    except:
        result = field
    return result

In [125]:
pat_hashtags = re.compile(r"#(\w+)")
def remove_hashtags(field):
    description = str(field).replace('\n', ' ').replace('description_end', ' ').strip().lower()

    # remove hashtags
    hashtags = list(set(pat_hashtags.findall(description)))
    hashtags = sorted(hashtags, key=lambda x: len(x), reverse=True)
    for hashtag in hashtags:
        description = description.replace('#' + hashtag, '')

    return description.replace('#', '').strip()

In [126]:
italian_stemmer = SnowballStemmer('italian')
def clean_text_aslist(field):
    description = remove_hashtags(field)

    token_list = []
    doc = nlp.tokenizer(description.replace("'", ' ').replace("’", ' ').lower())
    for token in doc:
        if not token.is_stop \
                and not token.is_punct \
                and not str(token) in stopwords \
                and not str(token).isdigit() \
                and not len(str(token).strip()) < 2:
            token_list.append(italian_stemmer.stem(str(token)))

    return token_list

In [127]:
def clean_text(df):
    df["title_processed"] = df["title"].apply(lambda x: demojize(x))
    df["description_processed"] = df["description"].apply(lambda x: demojize(x))
    # Remove Verbs from description
    schede_description = df.apply(
        lambda row: remove_verbs_adj(row['description']),
        axis=1)

    # Remove hastags, punctuation, stop words and numbers
    schede_description = schede_description.apply(
        lambda row: clean_text_aslist(row)) 
        
    # Remove Verbs from title
    schede_title = df.apply(
        lambda row: remove_verbs_adj(row['title']),
        axis=1)
        
    # Remove hastags, punctuation, stop words and numbers
    schede_title = schede_title.apply(
        lambda row: clean_text_aslist(row))
    
    df["description_processed"] = schede_description
    df["title_processed"] = schede_title

In [128]:
clean_text(df_categories)

In [129]:
df_categories

,id,userId,title,description,duration,datePublishing,creationDate,categories,viewCounter,,...,eventi culinari,food,food all'aperto,food drink,sport,tempo libero,travel,viaggi,title_processed,description_processed
0,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,[Viaggi],18,0,...,0,0,0,0,0,0,0,1,"[big, buddh]","[met, turist, assalt, spiagg, acqua, \n\n, mer..."
1,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,[Viaggi],16,0,...,0,0,0,0,0,0,0,1,"[wat, chalong, temp, phuket]","[spiagg, pen, temp, wat, chalong, big, buddh, ..."
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,[Viaggi],426,0,...,0,0,0,0,0,0,0,1,[tigr],"[marc, maltratt, animal, andar, sconsigl, pass..."
3,624e5982ec9ff859ff529ce8,571,"Yangon, la capitale storica del Myanmar",Probabilmente Yangon sarà il punto d’inizio se...,0,2022-04-07T03:24:51.101Z,2022-04-07T03:24:50.000+00:00,[Viaggi],6,0,...,0,0,0,0,0,0,0,1,"[capital, myanm]","[punt, iniz, viagg, myanm, \r\n, preg, difett,..."
4,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,18000000,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,[Viaggi],424,0,...,0,0,0,0,0,0,0,1,[beac],"[spiagg, isol, phuket, freedom, beac, spiagg, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,6213b3460f41f86438b07d9c,429,"Machu Picchu, meraviglia del mondo","È l’alba, ma la realtà è che in questo viaggio...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,[Viaggi],47,0,...,0,0,0,0,0,0,0,1,"[picchu, meravigl]","[alba, realt, viagg, rumor, casc, capell, fium..."
5720,61dd867f9c823e129b1f2cc2,429,I segreti di Moray,Sembra la prova schiacciante dell’esistenza de...,7200000,2022-01-11T13:30:45.997Z,2022-01-11T13:30:39.000+00:00,[Aria aperta],95,0,...,0,0,0,0,0,0,0,0,"[segret, moray]","[prov, esistt, alien, realt, stud, terrazz, mo..."
5721,61e0146111b3bf4d3185ad5d,427,"Machu Picchu, tra storia e meraviglia",Sono passati dieci (!) anni dalla mia esperien...,16200000,2022-01-13T12:04:35.591Z,2022-01-13T12:00:33.000+00:00,[Cultura],63,0,...,0,0,0,0,0,0,0,0,"[picchu, stor, meravigl]","[diec, esperit, machu, ricord, memor, sit, mer..."
5722,6273a0a9dd67f6198e5ac192,606,Come arrivare a Machu Picchu: modalità e tempi,Sembrerà un argomento banale: vuoi che sia dif...,0,2022-05-05T10:02:17.841Z,2022-05-05T10:02:17.000+00:00,[Viaggi],7,0,...,0,0,0,0,0,0,0,1,"[machu, picchu, modal]","[argoment, meravigl, ebben, realt, \r\n, inca,..."


### TFIDFVectorizer

In [130]:
tfidf_vectorizer = TfidfVectorizer(lowercase=False,
                                        min_df=5,
                                        max_df=0.10)
# description and title are put together
text = df_categories['description_processed'] + df_categories['title_processed']
t = text.apply(lambda x : " ".join(x))
tfidf_schede = tfidf_vectorizer.fit_transform(t)
dense_text = tfidf_schede.todense()
denselist_text = dense_text.tolist()

df_tfidf = pd.DataFrame(denselist_text, index=df_categories.index,
                                  columns=tfidf_vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [131]:
importance = np.argsort(np.asarray(df_tfidf.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())
top = tfidf_feature_names[importance[:20]]
for t in top:
  print(t)

isol
spiagg
lag
pizz
borg
sentier
mont
vin
palazz
giardin
pesc
vill
de
quart
castell
oper
chies
sant
vall
qualit


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


##Dates

In [132]:
def preprocess_dates(df):
    # Convert duration from ms to minutes
    df['duration_min'] = df.apply(lambda row: row["duration"] / 60000, axis=1)

    # Compute age of entry in days from today
    df['days'] = 0
    df.days = (pd.to_datetime("now").tz_localize('UTC') - pd.to_datetime(
        df["creationDate"])) // np.timedelta64(1, 'D')

In [133]:
preprocess_dates(df_categories)

In [134]:
df_categories

,id,userId,title,description,duration,datePublishing,creationDate,categories,viewCounter,,...,food all'aperto,food drink,sport,tempo libero,travel,viaggi,title_processed,description_processed,duration_min,days
0,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,[Viaggi],18,0,...,0,0,0,0,0,1,"[big, buddh]","[met, turist, assalt, spiagg, acqua, \n\n, mer...",0.0,97
1,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,[Viaggi],16,0,...,0,0,0,0,0,1,"[wat, chalong, temp, phuket]","[spiagg, pen, temp, wat, chalong, big, buddh, ...",0.0,97
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,[Viaggi],426,0,...,0,0,0,0,0,1,[tigr],"[marc, maltratt, animal, andar, sconsigl, pass...",180.0,129
3,624e5982ec9ff859ff529ce8,571,"Yangon, la capitale storica del Myanmar",Probabilmente Yangon sarà il punto d’inizio se...,0,2022-04-07T03:24:51.101Z,2022-04-07T03:24:50.000+00:00,[Viaggi],6,0,...,0,0,0,0,0,1,"[capital, myanm]","[punt, iniz, viagg, myanm, \r\n, preg, difett,...",0.0,78
4,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,18000000,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,[Viaggi],424,0,...,0,0,0,0,0,1,[beac],"[spiagg, isol, phuket, freedom, beac, spiagg, ...",300.0,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,6213b3460f41f86438b07d9c,429,"Machu Picchu, meraviglia del mondo","È l’alba, ma la realtà è che in questo viaggio...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,[Viaggi],47,0,...,0,0,0,0,0,1,"[picchu, meravigl]","[alba, realt, viagg, rumor, casc, capell, fium...",360.0,122
5720,61dd867f9c823e129b1f2cc2,429,I segreti di Moray,Sembra la prova schiacciante dell’esistenza de...,7200000,2022-01-11T13:30:45.997Z,2022-01-11T13:30:39.000+00:00,[Aria aperta],95,0,...,0,0,0,0,0,0,"[segret, moray]","[prov, esistt, alien, realt, stud, terrazz, mo...",120.0,164
5721,61e0146111b3bf4d3185ad5d,427,"Machu Picchu, tra storia e meraviglia",Sono passati dieci (!) anni dalla mia esperien...,16200000,2022-01-13T12:04:35.591Z,2022-01-13T12:00:33.000+00:00,[Cultura],63,0,...,0,0,0,0,0,0,"[picchu, stor, meravigl]","[diec, esperit, machu, ricord, memor, sit, mer...",270.0,162
5722,6273a0a9dd67f6198e5ac192,606,Come arrivare a Machu Picchu: modalità e tempi,Sembrerà un argomento banale: vuoi che sia dif...,0,2022-05-05T10:02:17.841Z,2022-05-05T10:02:17.000+00:00,[Viaggi],7,0,...,0,0,0,0,0,1,"[machu, picchu, modal]","[argoment, meravigl, ebben, realt, \r\n, inca,...",0.0,50


In [135]:
df_categories.to_csv(data_path, index=False)

#Save results

In [136]:
data = pd.read_csv(data_path)  

In [137]:
data

,id,userId,title,description,duration,datePublishing,creationDate,categories,viewCounter,Unnamed: 9,...,food all'aperto,food drink,sport,tempo libero,travel,viaggi,title_processed,description_processed,duration_min,days
0,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,['Viaggi'],18,0,...,0,0,0,0,0,1,"['big', 'buddh']","['met', 'turist', 'assalt', 'spiagg', 'acqua',...",0.0,97
1,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,['Viaggi'],16,0,...,0,0,0,0,0,1,"['wat', 'chalong', 'temp', 'phuket']","['spiagg', 'pen', 'temp', 'wat', 'chalong', 'b...",0.0,97
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,['Viaggi'],426,0,...,0,0,0,0,0,1,['tigr'],"['marc', 'maltratt', 'animal', 'andar', 'scons...",180.0,129
3,624e5982ec9ff859ff529ce8,571,"Yangon, la capitale storica del Myanmar",Probabilmente Yangon sarà il punto d’inizio se...,0,2022-04-07T03:24:51.101Z,2022-04-07T03:24:50.000+00:00,['Viaggi'],6,0,...,0,0,0,0,0,1,"['capital', 'myanm']","['punt', 'iniz', 'viagg', 'myanm', '\\r\\n', '...",0.0,78
4,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,18000000,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,['Viaggi'],424,0,...,0,0,0,0,0,1,['beac'],"['spiagg', 'isol', 'phuket', 'freedom', 'beac'...",300.0,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,6213b3460f41f86438b07d9c,429,"Machu Picchu, meraviglia del mondo","È l’alba, ma la realtà è che in questo viaggio...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,['Viaggi'],47,0,...,0,0,0,0,0,1,"['picchu', 'meravigl']","['alba', 'realt', 'viagg', 'rumor', 'casc', 'c...",360.0,122
5720,61dd867f9c823e129b1f2cc2,429,I segreti di Moray,Sembra la prova schiacciante dell’esistenza de...,7200000,2022-01-11T13:30:45.997Z,2022-01-11T13:30:39.000+00:00,['Aria aperta'],95,0,...,0,0,0,0,0,0,"['segret', 'moray']","['prov', 'esistt', 'alien', 'realt', 'stud', '...",120.0,164
5721,61e0146111b3bf4d3185ad5d,427,"Machu Picchu, tra storia e meraviglia",Sono passati dieci (!) anni dalla mia esperien...,16200000,2022-01-13T12:04:35.591Z,2022-01-13T12:00:33.000+00:00,['Cultura'],63,0,...,0,0,0,0,0,0,"['picchu', 'stor', 'meravigl']","['diec', 'esperit', 'machu', 'ricord', 'memor'...",270.0,162
5722,6273a0a9dd67f6198e5ac192,606,Come arrivare a Machu Picchu: modalità e tempi,Sembrerà un argomento banale: vuoi che sia dif...,0,2022-05-05T10:02:17.841Z,2022-05-05T10:02:17.000+00:00,['Viaggi'],7,0,...,0,0,0,0,0,1,"['machu', 'picchu', 'modal']","['argoment', 'meravigl', 'ebben', 'realt', '\\...",0.0,50


<h3>Preprocess using functions in support</h3>

In [138]:
#analyze.analyze_dataset(schede_path, data_analyzed_path, [])